In [2]:
from scipy import *
import numpy as np
import FNC

# Example 1.1.2

Recall the grade-school approximation to the number $\pi$.

$$
f(x) = \sin(x^2)
$$

In [2]:
p = 22/7
print(p)

3.142857142857143


Note that not all the digits displayed for `p` are the same as for $\pi$. As an approximation, its absolute and relative accuracy are

In [3]:
from numpy import pi
print("absolute accuracy: ",abs(p-pi))

absolute accuracy:  0.0012644892673496777


In [4]:
print("relative accuracy: ",abs(p-pi)/pi)
print("accurate digits: ",-np.log10(abs(p-pi)/pi))

relative accuracy:  0.0004024994347707008
accurate digits:  3.3952347251747166


# Example 1.1.3

There is no double precision number between $1$ and $1+\varepsilon_\text{mach}$. Thus the following difference is zero despite its appearance.

In [24]:
finfo = np.finfo(np.float32)

In [25]:
finfo.eps

1.1920929e-07

In [19]:
finfo.dtype

dtype('float64')

In [6]:
eps = np.finfo(float).eps
eps

2.220446049250313e-16

In [7]:
e = eps/2
e

1.1102230246251565e-16

In [9]:
1.0 + eps

1.0000000000000002

In [10]:
1.0 + eps/2 

1.0

In [12]:
(1.0 + eps/2) - 1.0 == eps/2

False

However, $1-\varepsilon_\text{mach}/2$ is a double precision number, so it and its negative are represented exactly:

In [14]:
1.0 + (eps/2 - 1.0) == eps/2

True

This is now the "correct" result. But we have found a rather shocking breakdown of the associative law of addition!

# Example 1.3.2

Here we show how to use `horner` to evaluate a polynomial. We first define a vector of the coefficients of $p(x)=(x−1)^3=x^3−3x^2+3x−1$, in descending degree order. Note that the textbook's functions are all in a namespace called `FNC`, to help distinguish them from other Python commands and modules.


In [3]:
c = np.array([1,-3,3,-1])
print( FNC.horner(c,1.6) )

0.21600000000000041


In [4]:
(1.6 - 1)**3

0.2160000000000001

The above is the value of $p(1.6)$, up to a rounding error.

# Example 1.3.3

Our first step is to construct a polynomial with six known roots.

In [6]:
help(np.poly)

Help on function poly in module numpy:

poly(seq_of_zeros)
    Find the coefficients of a polynomial with the given sequence of roots.
    
    .. note::
       This forms part of the old polynomial API. Since version 1.4, the
       new polynomial API defined in `numpy.polynomial` is preferred.
       A summary of the differences can be found in the
       :doc:`transition guide </reference/routines.polynomials>`.
    
    Returns the coefficients of the polynomial whose leading coefficient
    is one for the given sequence of zeros (multiple roots must be included
    in the sequence as many times as their multiplicity; see Examples).
    A square matrix (or array, which will be treated as a matrix) can also
    be given, in which case the coefficients of the characteristic polynomial
    of the matrix are returned.
    
    Parameters
    ----------
    seq_of_zeros : array_like, shape (N,) or (N, N)
        A sequence of polynomial roots, or a square array or matrix object.
    
  

In [17]:
r = np.array([-2.0,-1,1,1,3,6])
p = np.poly(r)
print(p)

[  1.  -8.   6.  44. -43. -36.  36.]


Now we use a standard numerical method for finding those roots, pretending that we don't know them already.

In [18]:
px = np.poly1d(p)
px

poly1d([  1.,  -8.,   6.,  44., -43., -36.,  36.])

In [19]:
px.roots

array([ 6.        ,  3.        , -2.        , -1.        ,  0.99999999,
        1.00000001])

In [20]:
r_computed = np.sort(px.roots)
print(r_computed)

[-2.         -1.          0.99999999  1.00000001  3.          6.        ]


Here are the relative errors in each of the computed roots.

In [21]:
print(abs(r - r_computed) / abs(r))

[3.33066907e-16 2.22044605e-16 1.23423448e-08 1.23423434e-08
 1.18423789e-15 1.18423789e-15]


It seems that the forward error is acceptably close to machine epsilon for double precision in all cases except the double root at $x=1$. This is not a surprise, though, given the poor conditioning at such roots.

Let's consider the backward error. The data in the rootfinding problem are the polynomial coefficients. We can apply poly to find the coefficients of the polynomial (that is, the data) whose roots were actually computed by the numerical algorithm.

In [22]:
p_computed = np.poly(r_computed)
print(p_computed)

[  1.  -8.   6.  44. -43. -36.  36.]


In [24]:
p_computed[1]

-7.999999999999996

In [25]:
p_computed[2]

5.999999999999998

In [26]:
p[1]

-8.0

In [27]:
p[2]

6.0

We find that in a relative sense, these coefficients are very close to those of the original, exact polynomial:

In [23]:
print(abs(p-p_computed)/abs(p))

[0.00000000e+00 5.55111512e-16 2.96059473e-16 4.84460956e-16
 1.65242497e-15 1.97372982e-16 1.57898386e-15]


In summary, even though there are some computed roots relatively far from their correct values, they are nevertheless the roots of a polynomial that is very close to the original.

# Example 1.3.4

In [29]:
a = 1.0
b = -(1e6+1e-6)
c = 1.0

In [30]:
x1 = (-b + np.sqrt(b*b-4*a*c)) / (2*a)
print(x1)

1000000.0


So far, so good. But:

In [31]:
x2 = (-b - np.sqrt(b*b-4*a*c)) / (2*a)
print(x2)

1.00000761449337e-06


The first value is correct to all stored digits, but the second has fewer than six accurate digits:

In [16]:
print( -np.log10(abs(1e-6-x2)/1e-6 ) )

5.118358987126217


In [32]:
-b

1000000.000001

In [33]:
np.sqrt(b*b - 4*a*c)

999999.999999

In [34]:
-b - np.sqrt(b*b-4*a*c)

2.00001522898674e-06

# Example 1.3.5

In [17]:
a = 1.0
b = -(1e6+1e-6)
c = 1.0

First, we find the "good" root using the quadratic forumla. 

In [18]:
x1 = (-b + np.sqrt(b*b-4*a*c)) / (2*a)
print(x1)

1000000.0


Then we use the alternative formula for computing the other root. 

In [19]:
x2 = c/(a*x1)
print(x2 - 1e-6)

0.0
